# Исследование надёжности заёмщиков

## Первый взгляд на данные

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

In [2]:
data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')
data.info()
data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


__Вывод:__
- Некоторые категориальные переменные (education, family_status, income_type, purpose) могут быть записаны как в нижнем, так и верхнем регистре. Необходимо привести текстовый формат к единому виду
- В наборе данных присутствуют пропуски

## Предобработка данных

### Обработка пропусков

Из общей информации о данных видно, что количество пропусков в столбцах 'total_income' и 'days_employed' совпадает.
Проверим, в одних ли и тех же строчках отсутствуют данные


In [3]:
empty = data[(data['total_income'].isnull()) & (data['days_employed'].isnull())]
empty.shape

(2174, 12)

Получается, здесь мы столкнусь со случайными пропусками в количественных переменных

Перед тем как обрабатывать пропуски проверим наличие артефактов

In [4]:
print("Уникальные значения в столбце 'children':", data['children'].unique())

print('Максимальный трудовой стаж:', data['days_employed'].max())

Уникальные значения в столбце 'children': [ 1  0  3  2 -1  4 20  5]
Максимальный трудовой стаж: 401755.40047533


Убираем минусы в столбцах `children` и `days_employed`. Очевидно, что человек не мог работать 4 тыс. дней. Возьмем за максимум 60 лет, т.е. 21900 дней.

In [5]:
data['children'] = data['children'].abs()
data['days_employed'] = data['days_employed'].abs()

In [6]:
data['days_employed'] = data['days_employed'].apply(lambda x: 21900 if x > 21900 else x)

Приведем текстовые переменные в нижний регистр.

In [7]:
cols_txt = ['education', 'family_status', 'income_type', 'purpose']
for i in cols_txt:
    data[i] = data[i].str.lower()

Общий размер заработной платы зависит от уровня образования человека, 
поэтому пропущенные значения в изначальном датафрейме будем заменять на медианные значения заработной платы по уровню образования.

In [8]:
data['total_income'] = data.groupby('education')['total_income'].apply(lambda x: x.fillna(x.median()))

Пропущенные значения в `days_employed` будем заменять на медианные значения `days_employed`, которые были сгруппированы по возрасту клиента.

In [9]:
data['days_employed'] = data.groupby('dob_years')['days_employed'].apply(lambda x: x.fillna(x.median()))

__Вывод__
- Пропущенные значения были обработы. Для замены пропущенных значений в столбце `days_employed` были использованы медианные значения, которые были найдены для каждой группы людей с определенным возрастом, а для `total_income` использованы медианные значения для каждого уровня образования.
- Причинами пропусков могут быть ошибки при вводе данных или при их выгрузке. У пенсинеров пропуски возможно вызвало осутствие дохода

### Замена типа данных

In [10]:
data['days_employed'] = data['days_employed'].astype(int)
data['total_income'] = data['total_income'].astype(int)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int32 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int32 
 11  purpose           21525 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 1.8+ MB


__Вывод__
- В столбцах `total_income` и `days_employed` заменили тип данных на целочисленный.

### Обработка дубликатов

In [11]:
print('Количество полных дубликатов до обработки:', data.duplicated().sum())

data = data.drop_duplicates().reset_index(drop=True)

print('Количество полных дубликатов после обработки:', data.duplicated().sum())

Количество полных дубликатов до обработки: 71
Количество полных дубликатов после обработки: 0


__Вывод:__
- С помощью метода drop_duplicates избавляемся от полных дубликатов. В ручном поиске дубликатов нет необходимости, так как здесь, если хоть одна характеристика отличается, мы имеет дело с другим клиентом. Причиной дубликатов возможно стало то, что одно и то же наблюдение случайно было введено более одного раза.

### Лемматизация

Посмотрим как часто отдельно взятая лемма встречается в датафрейме.

In [12]:
m = Mystem()
lemmas = m.lemmatize(' '.join(data['purpose']))
print(Counter(lemmas))

Counter({' ': 55023, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'подержать': 853, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'приобретение': 461, 'профильный': 436, 'подержанный': 111, '\n': 1})


In [13]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [ ]:
m = Mystem()


def lemmatize(words):
    
    '''Функция для лемматизации текста'''
    
    lemmas = m.lemmatize(words)
    lemmas = ' '.join(lemmas)
    return lemmas


data['lemmas_purpose'] = data['purpose'].apply(lemmatize)
data.head(10)

**Вывод**

- Для лемматизации была использована библиотека pymystem3. С помощью функции lemmatize и метода apply привели слова к их словарной форме. Лемматизированные слова для наглядности сохранили в новом столбце 'lemmas_purpose'.

### Категоризация данных

Для определения зависимости между наличием детей и возратом кредита срок составим датафрейм с необходимыми данными

In [ ]:
children = data[['children', 'debt']].reset_index(drop=True)
children['having_children'] = children['children'].apply(lambda x: 'есть дети' if x != 0 else 'нет детей')
children.head(10)

In [ ]:
children = data[['children', 'debt']].reset_index(drop=True)


def having_children(child):
    if child != 0:
        return 'есть дети'
    else:
        return 'нет детей'
    
    
children['having_children'] = children['children'].apply(lambda x: 'есть дети' if x != 0 else 'нет детей')
children.head(10)

Составляем таблицу для дальнейшего определения зависимости между целями взятия кредита и наличием задолжности

In [ ]:
data_purposes = data[['lemmas_purpose', 'debt']].reset_index(drop=True)

list_purposes = ['автомобиль', 'жилье', 'недвижимость', 'свадьба', 'образование']

Функция проходит по списку list_purposes. Проверяет присутствует ли значение из списка в ячейке датафрейма. 
Если да, возвращает значение из списка

In [ ]:
def lemmas_purpose(purpose):
    
    '''Функция проходит по списку list_purposes. 
    Проверяет присутствует ли значение из списка в ячейке датафрейма. 
    Если да, возвращает значение из списка'''
    
    for i in range(len(list_purposes)):
        if list_purposes[i] in purpose:
            purpose = list_purposes[i]
            return purpose

        
data_purposes['lemmas_purpose'] = data_purposes['lemmas_purpose'].apply(lemmas_purpose)
data_purposes.head(10)

Категоризация данных для дальнешнего определения зависимости между доходом и задолжностью

In [ ]:
income = data[['total_income', 'debt']]

Чтобы сгруппировать данные по уровню дохода, определим по каким значениям их делить

In [ ]:
income = income.sort_values(by='total_income').reset_index(drop=True)

Всего количество строк в датафрейме:

In [ ]:
number_of_rows = income.shape[0]

In [ ]:
part = number_of_rows // 5

first = part
second = part*2
third = part*3
fourth = part*4

income_min = income['total_income'].min()
income_max = income['total_income'].max()


def income_group(money):
    
    '''Функция возвращает диапазон дохода, используя нижеперечисленные правила'''
    
    if money <= income.iloc[first, 0]:
        money = '['+ str(income_min) + ',' + str(income.iloc[first, 0]) + ']'
        return money
    elif money <= income.iloc[second, 0]:
        money = '(' + str(income.iloc[first, 0]) + ',' + str(income.iloc[second, 0]) + ']'
        return money
    elif money <= income.iloc[third, 0]:
        money = '(' + str(income.iloc[second, 0]) + ',' + str(income.iloc[third, 0]) + ']'
        return money
    elif money <= income.iloc[fourth, 0]:
        money = '(' + str(income.iloc[third, 0]) + ',' + str(income.iloc[fourth, 0]) + ']'
        return money
    else:
        money = '(' + str(income.iloc[fourth, 0]) + ',' + str(income_max) + ']'
        return money

    
income['income_group'] = income['total_income'].apply(income_group)
income.head(10)

In [ ]:
income_new = data[['total_income', 'debt']].reset_index(drop=True)
income_new['total_income'] = pd.qcut(income_new['total_income'], 5, 
                                     labels=['низкий', 'ниже среднего', 'средний', 'выше среднего', 'высокий'])

income_new.head(10)


**Вывод**

Была проведена категоризация данных, необходимая для дальнейшего исследования.

## Ответы на вопросы

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
children_pivot = pd.pivot_table(children, index='having_children', values='debt', aggfunc=['count', 'sum', 'mean'])
children_pivot.style.format({('mean', 'debt'): '{:.2%}'})

<div class="alert alert-success">
<h2> Комментарий ревьюера v2 <a class="tocSkip"> </h2>

У тебя отличная работа!
    
Может тебе будет интересно: для улучшения визуализации датафрейма, можно пользоваться [style](https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html) библиотеки `pandas`. Там много прикольных штук для того, чтобы сделать датафрейм красивее 😊
Например, в нашем случае, чтобы выводить результат в процентах, а не в долях, можно прописать следующее:
    
`children_pivot.style.format({('mean', 'debt'): '{:.2%}'})`  
    
</div>

**Вывод**

Клиенты без детей реже имеют задолжность по возрату кредита по сравнению с человеком, у которого уже есть ребенок

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Доля клиентов с задолженностью в зависимости от семейного положения

In [ ]:
family_status_groupby = data.groupby('family_status').agg({'debt': ['count', 'sum', 'mean']})
family_status_groupby

In [ ]:
family_status_pivot = pd.pivot_table(data, index='family_status', values='debt', 
                                     aggfunc=['count', 'sum', 'mean'])
family_status_pivot

**Вывод**

Люли с семейным положением не женаты/не замужем и состоящие в гражданском браке чаще имеют задолжность.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Доля клиентов с задолженностью в зависимости от дохода

In [ ]:
income_new_groupby = income_new.groupby('total_income').agg({'debt': ['count', 'sum', 'mean']})
income_new_groupby

In [ ]:
income_new_pivot = pd.pivot_table(income_new, index='total_income', values='debt', 
                                     aggfunc=['count', 'sum', 'mean'])
income_new_pivot

**Вывод**

Больше всего задолжностей имеют люди со средним доходом, а меньше всего клиенты с самым высоким доходом

- Как разные цели кредита влияют на его возврат в срок?

Доля клиентов с задолженностью в зависимости от цели взятия кредита

In [ ]:
p_groupby = data_purposes.groupby('lemmas_purpose').agg({'debt': ['count', 'sum', 'mean']})
p_groupby

In [ ]:
p_pivot = pd.pivot_table(data_purposes, index='lemmas_purpose', values='debt', aggfunc=['count', 'sum', 'mean'])
p_pivot

**Вывод**

Люди, у которых цель взятия кредита была жилье/недвижимость реже имели проблемы с выплатой долга

## Шаг 4. Общий вывод

- В ходе предобработки данных были обнаружены пропуски в столбцах 'days_employed' и 'total_income', для заполнения которых была использована медиана. Артефакты были обработаны. Полные дубликаты строк удалены.   
- Для проведения исследования по определнию зависимости между наличием задолжности и другими параметрами, была произведена категоризация данных.  После некоторых расчетов сделан вывод, что на факт наличия задолжности по возврату кредита сильно влияет доход клиентов. Цель кредита, наличие детей и семейное положение тоже оказывает влияние, но меньше.